In [85]:
import streamlit as st
import pandas as pd
import requests
from datetime import datetime
from io import StringIO

from modules.get_elo_data import get_elo_data

In [ ]:
# Page config
st.set_page_config(page_title="Czech Football Clubs ELO", layout="wide")


In [134]:
club_mapping = {
    62: {"tm_name": "sk-slavia-prag", "elo_api_name": "SlaviaPraha", "club_name": "SK Slavia Praha"},
    5546: {"tm_name": "fk-mlada-boleslav", "elo_api_name": "MladaBoleslav", "club_name": "FK Mladá Boleslav"},
    715: {"tm_name": "fc-bohemians-prag-1905", "elo_api_name": "BohemiansPraha", "club_name": "Bohemians 1905"},
    1496: {"tm_name": "fk-pardubice", "elo_api_name": "Pardubice", "club_name": "FK Pardubice"},
    377: {"tm_name": "fc-banik-ostrau", "elo_api_name": "BanikOstrava", "club_name": "FC Baník Ostrava"},
    197: {"tm_name": "ac-sparta-prag", "elo_api_name": "SpartaPraha", "club_name": "AC Sparta Praha"},
    1322: {"tm_name": "fk-jablonec", "elo_api_name": "Jablonec", "club_name": "FK Jablonec"},
    1897: {"tm_name": "fc-hradec-kralove", "elo_api_name": "HradecKralove", "club_name": "FC Hradec Králové"},
    13726: {"tm_name": "mfk-karvina", "elo_api_name": "Karvina", "club_name": "MFK Karviná"},
    1321: {"tm_name": "sk-dynamo-ceske-budejovice", "elo_api_name": "CeskeBudejovice", "club_name": "SK Dynamo České Budějovice"},
    814: {"tm_name": "fk-teplice", "elo_api_name": "Teplice", "club_name": "FK Teplice"},
    697: {"tm_name": "fc-slovan-liberec", "elo_api_name": "SlovanLiberec", "club_name": "FC Slovan Liberec"},
    941: {"tm_name": "fc-viktoria-pilsen", "elo_api_name": "ViktoriaPlzen", "club_name": "FC Viktoria Plzeň"},
    2311: {"tm_name": "sk-sigma-olmutz", "elo_api_name": "SigmaOlomouc", "club_name": "SK Sigma Olomouc"},
    450: {"tm_name": "fk-dukla-prag", "elo_api_name": "Dukla", "club_name": "FK Dukla Praha"},
    5544: {"tm_name": "1-fc-slovacko", "elo_api_name": "Slovacko", "club_name": "1. FC Slovácko"}
}

# Convert to DataFrame if needed
import pandas as pd
df = pd.DataFrame.from_dict(club_mapping, orient="index").reset_index().rename(columns={"index": "club_id"})

# Display DataFrame
print(df)


czech_clubs = pd.DataFrame.from_dict(club_mapping, orient="index").reset_index()

# Rename columns
czech_clubs.rename(columns={"index": "club_id"}, inplace=True)

    club_id                     tm_name     elo_api_name  \
0        62              sk-slavia-prag      SlaviaPraha   
1      5546           fk-mlada-boleslav    MladaBoleslav   
2       715      fc-bohemians-prag-1905   BohemiansPraha   
3      1496                fk-pardubice        Pardubice   
4       377             fc-banik-ostrau     BanikOstrava   
5       197              ac-sparta-prag      SpartaPraha   
6      1322                 fk-jablonec         Jablonec   
7      1897           fc-hradec-kralove    HradecKralove   
8     13726                 mfk-karvina          Karvina   
9      1321  sk-dynamo-ceske-budejovice  CeskeBudejovice   
10      814                  fk-teplice          Teplice   
11      697           fc-slovan-liberec    SlovanLiberec   
12      941          fc-viktoria-pilsen    ViktoriaPlzen   
13     2311             sk-sigma-olmutz     SigmaOlomouc   
14      450               fk-dukla-prag            Dukla   
15     5544               1-fc-slovacko 

In [ ]:
def get_elo_data(club):
    try:
        """Fetch ELO data from the API"""
        response = requests.get(f'http://api.clubelo.com/{club}')
        df = pd.read_csv(StringIO(response.text))
        latest_elo = df.iloc[-1]['Elo']
        latest_date = df.iloc[-1]['From']
        return latest_elo, latest_date, df
    except:
        return None, None, None

In [101]:
# Create a list to store club data
df_club_elo = []

# Fetch data for each club
for club in czech_clubs["elo_api_name"]:
    elo, date, club_elo_df = get_elo_data(club)
    if elo is not None:
        # Format club name for display (remove camel case)
        display_name = ''.join([' ' + c if c.isupper() else c for c in club]).strip()
        df_club_elo.append({
            'Club': display_name,
            'ELO Rating': round(elo),
            'Last Updated': date
        })

df_club_elo = pd.DataFrame(df_club_elo)

czech_clubs["latest_elo"]

In [36]:

# App title
st.title("Czech Football Clubs ELO Ratings")

# Display stats
st.subheader("Current ELO Ratings")
st.dataframe(
    clubs_df,
    hide_index=True,
    column_config={
        'ELO Rating': st.column_config.NumberColumn(format="%d"),
        'Last Updated': st.column_config.DateColumn('Last Updated')
    }
)

# Calculate and display some statistics
st.subheader("League Statistics")
col1, col2, col3 = st.columns(3)

with col1:
    st.metric("Highest ELO", f"{clubs_df['ELO Rating'].max()}")
    st.caption(f"({clubs_df.iloc[0]['Club']})")
    
with col2:
    st.metric("Average ELO", f"{round(clubs_df['ELO Rating'].mean())}")
    
with col3:
    st.metric("Lowest ELO", f"{clubs_df['ELO Rating'].min()}")
    st.caption(f"({clubs_df.iloc[-1]['Club']})")

# Create a bar chart
st.subheader("ELO Ratings Comparison")
chart_data = clubs_df.set_index('Club')
st.bar_chart(chart_data['ELO Rating'])

2025-02-13 19:45:34.669 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 19:45:34.670 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 19:45:34.670 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 19:45:34.671 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 19:45:34.671 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 19:45:34.672 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 19:45:34.673 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 19:45:34.673 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

In [128]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

season = 2024

'''
def get_club_ids(season):
    clubs = pd.DataFrame(columns=["club_id", "club_name"])
    url = f"https://www.transfermarkt.com/1-fc-slovacko/spielplandatum/verein/5544/plus/0?saison_id={season}&wettbewerb_id=TS1"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Failed to retrieve data")
        exit()

    soup = BeautifulSoup(response.text, 'html.parser')

    soup = soup.find("div", class_="responsive-table")
    # Finding match rows in the fixture table
    match_rows = soup.select("tr")

    for row in match_rows:
        zentriert_columns = [td.text.strip() for td in row.find_all("td", class_=["zentriert", "no-border-links hauptlink"])]
        if not zentriert_columns:
            continue
        club_id = re.search(r"/verein/(\d+)/", [td.find("a")["href"] for td in row.find_all("td", class_="no-border-links hauptlink") if td.find("a")][0]).group(1)
        club_name = re.search(r"/([^/]+)/spielplan/", [td.find("a")["href"] for td in row.find_all("td", class_="no-border-links hauptlink") if td.find("a")][0]).group(1)
        new_row = pd.DataFrame({"club_id": [club_id], "club_name": [club_name]})
        clubs = pd.concat([clubs, new_row], ignore_index=True)

    clubs = clubs[['club_id', 'club_name']].drop_duplicates()
    return clubs

clubs = get_club_ids(season)


'''

fixtures = []
for index, row in czech_clubs.iterrows():
    club_id = row["club_id"]
    tm_name = row["tm_name"]
    club_name = row["club_name"]
    url = f"https://www.transfermarkt.de/{tm_name}/spielplandatum/verein/{club_id}/plus/0?saison_id=2024&wettbewerb_id=TS1"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("Failed to retrieve data")
        continue

    soup = BeautifulSoup(response.text, 'html.parser')
    

    soup = soup.find("div", class_="responsive-table")
    # Finding match rows in the fixture table
    match_rows = soup.select("tr")

    for row in match_rows:
        zentriert_columns = [td.text.strip() for td in row.find_all("td", class_=["zentriert", "no-border-links hauptlink"])]
        if not zentriert_columns:
            continue
        
        if zentriert_columns[3] == 'H':
            home_team = tm_name
            away_team = re.search(r"/([^/]+)/spielplan/", [td.find("a")["href"] for td in row.find_all("td", class_="no-border-links hauptlink") if td.find("a")][0]).group(1)
        
        else:
            home_team = re.search(r"/([^/]+)/spielplan/", [td.find("a")["href"] for td in row.find_all("td", class_="no-border-links hauptlink") if td.find("a")][0]).group(1)
            away_team = tm_name

        event_date = zentriert_columns[1]
        event_time = zentriert_columns[2]

        fixtures.append({"home_team": home_team, "away_team": away_team, "event_date": str(event_date), "event_time": event_time})
    

fixtures = pd.DataFrame(fixtures)
fixtures.drop_duplicates(inplace=True)






<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\d'
/var/folders/w2/bg2zc3bn6wd8hhlvl5x9v4t80000gp/T/ipykernel_90104/3984740181.py:8: SyntaxWarning: invalid escape sequence '\d'
  '''


In [129]:
fixtures["event_date"] = pd.to_datetime(fixtures["event_date"].str[4:], format="%d.%m.%y").dt.strftime("%Y-%m-%d")

fixtures["event_timestamp"] = pd.to_datetime(fixtures["event_date"] + " " + fixtures["event_time"], format="%Y-%m-%d %H:%M")

In [132]:
fixtures["home_team"].unique()

array(['1-fc-slovacko', 'sk-slavia-prag', 'fc-slovan-liberec',
       'fk-mlada-boleslav', 'fc-bohemians-prag-1905', 'fk-jablonec',
       'fc-hradec-kralove', 'sk-dynamo-ceske-budejovice',
       'fc-banik-ostrau', 'sk-sigma-olmutz', 'fk-teplice', 'fk-pardubice',
       'fc-viktoria-pilsen', 'ac-sparta-prag', 'fk-dukla-prag',
       'mfk-karvina'], dtype=object)

In [135]:
tm_to_club = {entry["tm_name"]: entry["club_name"] for entry in club_mapping.values()}
fixtures["home_team"] = fixtures["home_team"].map(tm_to_club)
fixtures["away_team"] = fixtures["away_team"].map(tm_to_club)



In [136]:
fixtures

,home_team,away_team,event_date,event_time,event_timestamp
0,1. FC Slovácko,SK Slavia Praha,2024-07-21,17:00,2024-07-21 17:00:00
1,SK Slavia Praha,SK Dynamo České Budějovice,2024-07-27,20:00,2024-07-27 20:00:00
2,FC Slovan Liberec,SK Slavia Praha,2024-08-02,17:30,2024-08-02 17:30:00
3,SK Slavia Praha,SK Sigma Olomouc,2024-08-10,20:00,2024-08-10 20:00:00
4,SK Slavia Praha,FK Teplice,2024-08-17,17:00,2024-08-17 17:00:00
...,...,...,...,...,...
404,SK Sigma Olomouc,1. FC Slovácko,2024-11-27,17:00,2024-11-27 17:00:00
412,FK Dukla Praha,SK Sigma Olomouc,2025-02-22,00:00,2025-02-22 00:00:00
418,1. FC Slovácko,SK Sigma Olomouc,2025-04-12,00:00,2025-04-12 00:00:00
436,FK Dukla Praha,1. FC Slovácko,2024-12-04,17:30,2024-12-04 17:30:00
